In [50]:
import os
import pandas as pd
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [51]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").config("spark.driver.memory", "2g").getOrCreate()

In [52]:
# Uploading the Genres file
genres_df = spark.read.csv("genres.csv", header=True, inferSchema=True)
genres_df.show()

+---+--------------------+---------+
|_c0|               title|    genre|
+---+--------------------+---------+
|  0|           Toy Story|Animation|
|  1|           Toy Story|   Comedy|
|  2|           Toy Story|   Family|
|  3|             Jumanji|Adventure|
|  4|             Jumanji|  Fantasy|
|  5|             Jumanji|   Family|
|  6|    Grumpier Old Men|  Romance|
|  7|    Grumpier Old Men|   Comedy|
|  8|   Waiting to Exhale|   Comedy|
|  9|   Waiting to Exhale|    Drama|
| 10|   Waiting to Exhale|  Romance|
| 11|Father of the Bri...|   Comedy|
| 12|                Heat|   Action|
| 13|                Heat|    Crime|
| 14|                Heat|    Drama|
| 15|                Heat| Thriller|
| 16|             Sabrina|   Comedy|
| 17|             Sabrina|  Romance|
| 18|        Tom and Huck|   Action|
| 19|        Tom and Huck|Adventure|
+---+--------------------+---------+
only showing top 20 rows



In [53]:
# Uploading the Movies file
movies_df = spark.read.csv("movies.csv", header=True, inferSchema=True)
movies_df.show()

+--------------------+------+---------+----+--------------------+-----+---------+------------------+--------------------+-----------------+--------------+---------+------------+--------------------+-------+
|                name|rating|    genre|year|            released|score|    votes|          director|              writer|             star|       country|   budget|       gross|             company|runtime|
+--------------------+------+---------+----+--------------------+-----+---------+------------------+--------------------+-----------------+--------------+---------+------------+--------------------+-------+
|         The Shining|     R|    Drama|1980|June 13, 1980 (Un...|  8.4| 927000.0|   Stanley Kubrick|        Stephen King|   Jack Nicholson|United Kingdom|    1.9E7| 4.6998772E7|        Warner Bros.|  146.0|
|     The Blue Lagoon|     R|Adventure|1980|July 2, 1980 (Uni...|  5.8|  65000.0|    Randal Kleiser|Henry De Vere Sta...|   Brooke Shields| United States|4500000.0| 5.88531

In [54]:
# Uploading the Metadata file
metadata_df = spark.read.csv("movies_metadata.csv", header=True, inferSchema=True)
metadata_df.show()

+-----+---------------------+--------+--------------------+--------------------+-----+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------+--------------------+-----------------+
|adult|belongs_to_collection|  budget|              genres|            homepage|   id|  imdb_id|original_language|      original_title|            overview|popularity|         poster_path|production_companies|production_countries|        release_date|             revenue|             runtime|    spoken_languages|  status|             tagline|               title|   video|        vote_average|       vote_count|
+-----+---------------------+--------+--------------------+--------------------+-----+---------+-----------------+--------------------+--------------------+----------+-----

In [55]:
# Creating Temp view for each of the tables.
genres_df.createOrReplaceTempView("genres")
movies_df.createOrReplaceTempView("movies")
metadata_df.createOrReplaceTempView("metadata")

In [56]:
budget_revenue_meta = spark.sql("""
  SELECT title, budget, revenue 
  FROM metadata 
  WHERE title is not null AND REGEXP_REPLACE(revenue, '[^0-9]', '') = revenue""")
budget_revenue_meta.show()

+--------------------+--------+---------+
|               title|  budget|  revenue|
+--------------------+--------+---------+
|           Toy Story|30000000|373554033|
|             Jumanji|65000000|262797249|
|    Grumpier Old Men|       0|        0|
|Father of the Bri...|       0| 76578911|
|                Heat|60000000|187436818|
|             Sabrina|58000000|        0|
|        Tom and Huck|       0|        0|
|        Sudden Death|35000000| 64350171|
|           GoldenEye|58000000|352194034|
|The American Pres...|62000000|107879496|
|Dracula: Dead and...|       0|        0|
|               Balto|       0| 11348324|
|               Nixon|44000000| 13681765|
|    Cutthroat Island|98000000| 10017322|
|              Casino|52000000|116112375|
|Sense and Sensibi...|16500000|135000000|
|          Four Rooms| 4000000|  4300000|
|Ace Ventura: When...|30000000|212385533|
|         Money Train|60000000| 35431113|
|             Copycat|       0|        0|
+--------------------+--------+---

In [57]:
budget_revenue_meta.createOrReplaceTempView("budget_revenue_meta")
spark.sql("SELECT * FROM budget_revenue_meta where revenue is null").show()


+-----+------+-------+
|title|budget|revenue|
+-----+------+-------+
+-----+------+-------+



In [58]:
movies_df.show(2)

+---------------+------+---------+----+--------------------+-----+--------+---------------+--------------------+--------------+--------------+---------+-----------+-----------------+-------+
|           name|rating|    genre|year|            released|score|   votes|       director|              writer|          star|       country|   budget|      gross|          company|runtime|
+---------------+------+---------+----+--------------------+-----+--------+---------------+--------------------+--------------+--------------+---------+-----------+-----------------+-------+
|    The Shining|     R|    Drama|1980|June 13, 1980 (Un...|  8.4|927000.0|Stanley Kubrick|        Stephen King|Jack Nicholson|United Kingdom|    1.9E7|4.6998772E7|     Warner Bros.|  146.0|
|The Blue Lagoon|     R|Adventure|1980|July 2, 1980 (Uni...|  5.8| 65000.0| Randal Kleiser|Henry De Vere Sta...|Brooke Shields| United States|4500000.0|5.8853106E7|Columbia Pictures|  104.0|
+---------------+------+---------+----+------

In [59]:
movies2 = spark.sql("""
          SELECT 
            movies.name,
            movies.rating,
            movies.genre,
            movies.year,
            movies.released,
            movies.score,
            movies.votes,
            movies.director,
            movies.writer,
            movies.star,
            movies.country,
            CASE 
              WHEN movies.budget is Null THEN 0
              ELSE budget
            END AS budget,
            CASE 
              WHEN movies.gross is Null THEN 0
              ELSE gross
            END AS gross,
            movies.company,
            movies.runtime
          FROM movies
          """)
movies2.createOrReplaceTempView("movies2")
spark.sql("select * from movies2").show()

+--------------------+------+---------+----+--------------------+------------------+--------------------+-----------------+--------------+---------+------------+--------------------+-------+
|                name|rating|    genre|year|            released|          director|              writer|             star|       country|   budget|       gross|             company|runtime|
+--------------------+------+---------+----+--------------------+------------------+--------------------+-----------------+--------------+---------+------------+--------------------+-------+
|         The Shining|     R|    Drama|1980|June 13, 1980 (Un...|   Stanley Kubrick|        Stephen King|   Jack Nicholson|United Kingdom|    1.9E7| 4.6998772E7|        Warner Bros.|  146.0|
|     The Blue Lagoon|     R|Adventure|1980|July 2, 1980 (Uni...|    Randal Kleiser|Henry De Vere Sta...|   Brooke Shields| United States|4500000.0| 5.8853106E7|   Columbia Pictures|  104.0|
|Star Wars: Episod...|    PG|   Action|1980|J

In [60]:
spark.sql("select count(year) from movies where budget is null").show()

+-----------+
|count(year)|
+-----------+
|       2171|
+-----------+



In [61]:
spark.sql("select count(year) from movies2 where budget == 0").show()

+-----------+
|count(year)|
+-----------+
|       2171|
+-----------+



In [62]:
new_df = spark.sql("""
          SELECT 
            movies2.name,
            movies2.rating,
            movies2.genre,
            movies2.year,
            movies2.released,
            movies2.director,
            movies2.writer,
            movies2.star,
            movies2.country,
            CASE 
              WHEN movies2.budget != 0 THEN movies2.budget
              ELSE budget_revenue_meta.budget
            END AS budget,
            CASE 
              WHEN movies2.gross != 0 THEN movies2.gross
              ELSE budget_revenue_meta.revenue
            END AS gross,
            movies2.company,
            movies2.runtime
          FROM movies2
          LEFT JOIN budget_revenue_meta
          ON movies2.name = budget_revenue_meta.title
          """)
new_df.show()

+--------------------+------+---------+----+--------------------+------------------+--------------------+-----------------+--------------+---------+------------+--------------------+-------+
|                name|rating|    genre|year|            released|          director|              writer|             star|       country|   budget|       gross|             company|runtime|
+--------------------+------+---------+----+--------------------+------------------+--------------------+-----------------+--------------+---------+------------+--------------------+-------+
|         The Shining|     R|    Drama|1980|June 13, 1980 (Un...|   Stanley Kubrick|        Stephen King|   Jack Nicholson|United Kingdom|    1.9E7| 4.6998772E7|        Warner Bros.|  146.0|
|     The Blue Lagoon|     R|Adventure|1980|July 2, 1980 (Uni...|    Randal Kleiser|Henry De Vere Sta...|   Brooke Shields| United States|4500000.0| 5.8853106E7|   Columbia Pictures|  104.0|
|     The Blue Lagoon|     R|Adventure|1980|J

In [63]:
new_df.createOrReplaceTempView("joined")
spark.sql("SELECT * FROM joined where budget == 0").show()

+--------------------+------+---------+----+--------------------+-----------------+------------------+--------------------+--------------+------+-----------+--------------------+-------+
|                name|rating|    genre|year|            released|         director|            writer|                star|       country|budget|      gross|             company|runtime|
+--------------------+------+---------+----+--------------------+-----------------+------------------+--------------------+--------------+------+-----------+--------------------+-------+
|                Fame|     R|    Drama|1980|May 16, 1980 (Uni...|      Alan Parker|  Christopher Gore|         Eddie Barth| United States|     0|2.1202829E7|Metro-Goldwyn-May...|  134.0|
|                Fame|     R|    Drama|1980|May 16, 1980 (Uni...|      Alan Parker|  Christopher Gore|         Eddie Barth| United States|     0|2.1202829E7|Metro-Goldwyn-May...|  134.0|
|          Stir Crazy|     R|   Comedy|1980|December 12, 1980...|

In [73]:
spark.sql("select count(name) from joined where budget == 0").show()

+-----------+
|count(name)|
+-----------+
|       1683|
+-----------+



In [76]:


movies_df = new_df.toPandas()
movies_df.to_csv("movies3.csv", index=False)